# Toxic Comment Classification

Discussing things you care about can be difficult. The threat of abuse and harassment online means that many people stop expressing themselves and give up on seeking different opinions. Platforms struggle to effectively facilitate conversations, leading many communities to limit or completely shut down user comments.

The Conversation AI team, a research initiative founded by Jigsaw and Google (both a part of Alphabet) are working on tools to help improve online conversation. One area of focus is the study of negative online behaviors, like toxic comments (i.e. comments that are rude, disrespectful or otherwise likely to make someone leave a discussion). So far they’ve built a range of publicly available models served through the Perspective API, including toxicity. But the current models still make errors, and they don’t allow users to select which types of toxicity they’re interested in finding (e.g. some platforms may be fine with profanity, but not with other types of toxic content).

In this competition, you’re challenged to build a multi-headed model that’s capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate better than Perspective’s current models. You’ll be using a dataset of comments from Wikipedia’s talk page edits. Improvements to the current model will hopefully help online discussion become more productive and respectful.

The dataset can be found on Kaggle : 

https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

# Libraries

In [2]:
import pandas as pd
import numpy as np
import re
import string
import itertools as it
import pickle
import os
from  pathlib import Path

import nltk
from nltk.corpus import stopwords                  # module for stop words that come with NLTK
from nltk.stem.wordnet import WordNetLemmatizer    # module for lemmatization
from nltk import word_tokenize, pos_tag            # tokenization and Part of Speech tagging

nltk.download('stopwords') #stopwords used to preprocess the corpus

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
stopwords_english = stopwords.words('english') # a list of English stopwords

# Lemmatizer = lemmatizer = WordNetLemmatizer()  # a method that returns the lemmatized form of word 
#                                                # ("was" => "be" - "rocks" => "rock")

# Import Data

## File descriptions
* train.csv - the training set, contains comments with their binary labels
* test.csv - the test set, you must predict the toxicity probabilities for these comments. To deter hand 
labeling, the test set contains some comments which are not included in scoring.
* sample_submission.csv - a sample submission file in the correct format
* test_labels.csv - labels for the test data; value of -1 indicates it was not used for scoring; (Note: file added after competition close!)

In [4]:
train_data = pd.read_csv("Data/train.csv")
test_data = pd.read_csv('Data/test.csv')
test_label_data = pd.read_csv('Data/test_labels.csv')

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [6]:
train_data.isna().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [7]:
train_data.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


As we can see the training dataset contains :
* the comment ID
* the raw text
* the different categories of toxicity

In [8]:
# Let's check some comments
for i in range(10):
    print(train_data['comment_text'][i])
    print('---------------')

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
---------------
D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)
---------------
Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
---------------
"
More
I can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any prefere

In [9]:
#Let's check in the test.csv
test_data.head(10)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,000247e83dcc1211,:Dear god this site is horrible.
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ..."
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...


Here we just have the ID's and comments with no classification

In [10]:
test_label_data.head(10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1
5,0001ea8717f6de06,0,0,0,0,0,0
6,00024115d4cbde0f,-1,-1,-1,-1,-1,-1
7,000247e83dcc1211,0,0,0,0,0,0
8,00025358d4737918,-1,-1,-1,-1,-1,-1
9,00026d1092fe71cc,-1,-1,-1,-1,-1,-1


This dataset is filled with ID and classification.  
A lot of rows are filled with -1, which means that these id's were not used for scoring (check Kaggle page for more information https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data)

In [11]:
# We are going to use this dataset to define the accuracy of our models
test_label_data = test_label_data.loc[test_label_data['toxic']!=-1]

In [12]:
# Let's calculate the % of toxic comments
test_label_data.iloc[:,1:-1].sum(axis=0) / test_label_data.shape[0]

toxic           0.095189
severe_toxic    0.005736
obscene         0.057692
threat          0.003298
insult          0.053565
dtype: float64

As we can see, there are a few of toxic comments compare to the dataset size.  
However it will be enough to test our model, because this dataset is different from the training set.

In [13]:
# Let's group comments and classification with ID's
test = test_label_data.merge(test_data, on='id', how="inner")
test.head(10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text
0,0001ea8717f6de06,0,0,0,0,0,0,Thank you for understanding. I think very high...
1,000247e83dcc1211,0,0,0,0,0,0,:Dear god this site is horrible.
2,0002f87b16116a7f,0,0,0,0,0,0,"""::: Somebody will invariably try to add Relig..."
3,0003e1cccfd5a40a,0,0,0,0,0,0,""" \n\n It says it right there that it IS a typ..."
4,00059ace3e3e9a53,0,0,0,0,0,0,""" \n\n == Before adding a new product to the l..."
5,000663aff0fffc80,0,0,0,0,0,0,this other one from 1897
6,000689dd34e20979,0,0,0,0,0,0,== Reason for banning throwing == \n\n This ar...
7,000844b52dee5f3f,0,0,0,0,0,0,|blocked]] from editing Wikipedia. |
8,00091c35fa9d0465,1,0,0,0,0,0,"== Arabs are committing genocide in Iraq, but ..."
9,000968ce11f5ee34,0,0,0,0,0,0,Please stop. If you continue to vandalize Wiki...


In [14]:
# Check if it worked
# We take id's from test df and we check comments in test_data
id = "000968ce11f5ee34"
test_data.loc[test_data["id"] == id]

,id,comment_text
22,000968ce11f5ee34,Please stop. If you continue to vandalize Wiki...


In [15]:
# Check if it worked
# We take id's from test df and we check comments in test_data
id = "000689dd34e20979"
test_data.loc[test_data["id"] == id]

,id,comment_text
17,000689dd34e20979,== Reason for banning throwing == \n\n This ar...


It worked

# Clean the corpus

In [16]:
# We define the list of punctuations we want to remove
# Note that we let ! in the corpus
# punc = '''()-[]{};:'"\,<>./?@#$%^&*_~'''

In [17]:
#Let''s define a function that preprocesses a text

def preprocess(corpus):
    
    '''
    From a string, make text lowercase, remove hyperlinks, punctuation, word containing numbers, stopwords.
    Input : a list of strings
    Output : a list of tokens stored in a generator (yield)
    '''

    for text in corpus:

        text = text.lower()                                               # Lowercase
        text = re.sub(r'https?://[^\s\n\r]+', '', text)                   # Remove links
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)   # Remove punctuation
        text = re.sub('\w*\d\w*', '', text)                               # Remove words containing numbers
    
        yield ' '.join([word for word in text.split(' ') if word not in stopwords_english]) # Return a generator 

In [18]:
%%time

# We save the cleaned comments in a list to be easily manipulated
clean_comments = list(preprocess(train_data['comment_text']))

Wall time: 41.7 s


In [19]:
for i in range(10):
    print(clean_comments[i])
    print('------------')

explanation
why edits made username hardcore metallica fan reverted werent vandalisms closure gas voted new york dolls fac please dont remove template talk page since im retired 
------------
daww matches background colour im seemingly stuck thanks  talk  january   utc
------------
hey man im really trying edit war guy constantly removing relevant information talking edits instead talk page seems care formatting actual info
------------

more
i cant make real suggestions improvement  wondered section statistics later subsection types accidents  think references may need tidying exact format ie date format etc later noone else first  preferences formatting style references want please let know

there appears backlog articles review guess may delay reviewer turns listed relevant form eg wikipediagoodarticlenominationstransport  
------------
sir hero chance remember page thats
------------


congratulations well use tools well  · talk 
------------
cocksucker piss around work
-----------

We note some words with no meaning, or typos. It can be better but we are going to work with that at first.



In [20]:
%%time
# We do the same for the test set
test_clean_comments = list(preprocess(test["comment_text"]))

Wall time: 20.9 s


# Word Embeddings

In this section, we will try out several word enbeddings methods. 

Let's use the simplest one : bag-of-words (BOW)

BOW method calculates the frequency of a word for each document, based on a Vocabulary.  
To use BOW, instead of recreating from scratch, we can use the library Scikit Learn => CountVectorizer

In [21]:
%%time

# Bag-of-words
vectorizer = CountVectorizer(min_df=3,max_df=0.9) #Filter words that are note present at least in min_df documents & no more that 90% of all documents
bow = vectorizer.fit_transform(clean_comments) #return a document-term matrix (n_samples,n_features)
bow_test = vectorizer.transform(test_clean_comments) # We do the same for test set, we just transform to have the same number of words

Wall time: 15.8 s


In [22]:
bow.shape , bow_test.shape

((159571, 52731), (63978, 52731))

We have generated a sparse matrix, composed of word frequencies for each document, with only a small number of non-zero elements (*stored elements* in the representation  below).  

*Note*  
It appears that using a generator (produced with yield in the preprocessing function *preprocess*) accelerate the preprocessing, but *fit_transform* actually takes more time this way.

In [23]:
# Let's take a look at the features / vocabulary
print(vectorizer.get_feature_names_out()[:30])
print('------------')
print(vectorizer.get_feature_names_out()[100:130])
print('------------')
print(vectorizer.get_feature_names_out()[1000:1030])
print('------------')
print(vectorizer.get_feature_names_out()[10000:10030])

['aa' 'aaa' 'aaand' 'aac' 'aachen' 'aah' 'aaliyah' 'aamir' 'aan' 'aand'
 'aang' 'aap' 'aaps' 'aar' 'aardvark' 'aarem' 'aaron' 'aarons' 'aas'
 'aatalk' 'aau' 'aave' 'ab' 'aba' 'aback' 'abad' 'abaddon' 'abandon'
 'abandoned' 'abandoning']
------------
['abolish' 'abolished' 'abolishing' 'abolition' 'abolitionist'
 'abolitionists' 'abomb' 'abominable' 'abomination' 'abominations'
 'aboriginal' 'aboriginals' 'aborigine' 'aborigines' 'abort' 'aborted'
 'abortion' 'abortions' 'abot' 'abotu' 'abou' 'aboumekhael' 'abound'
 'abounds' 'abour' 'about' 'aboutcom' 'abouth' 'abouti' 'above']
------------
['agreement' 'agreements' 'agrees' 'agress' 'agressing' 'agression'
 'agressive' 'agressively' 'agressor' 'agricultural' 'agriculture'
 'agriculturists' 'agrizoophobia' 'aground' 'ags' 'aguilera' 'aguri' 'agw'
 'ah' 'aha' 'ahaha' 'ahahahahaha' 'aharon' 'ahd' 'ahead' 'ahem' 'ahh'
 'ahhh' 'ahhhh' 'ahhrelief']
------------
['copright' 'coproduced' 'coproducer' 'cops' 'coptic' 'copts' 'copula'
 'copulat

As we can see, some words are not in the english dictionary, or are just grostesque.  
However filtering this word would filter "toxic" words too, as "cocksucker".  
We will therfore let these outliers in the dataset for the moment.  

# Models

## Train & Test Preparation 

In [33]:
# Let's define target, which is the classification made by human
target = train_data[['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']]
# target = np.array(target) #transform dataframe into array
target.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


Let's check if target values are balanced.   
In other words, is the target made of as much toxic as non-toxic comments

In [25]:
target.sum(axis=0) / target.shape[0]

toxic            0.095844
severe_toxic     0.009996
obscene          0.052948
threat           0.002996
insult           0.049364
identity_hate    0.008805
dtype: float64

As we can see, the target set  is not balanced.

In [35]:
keys = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
test[keys].sum(axis=0) /test.shape[0]

toxic            0.095189
severe_toxic     0.005736
obscene          0.057692
threat           0.003298
insult           0.053565
identity_hate    0.011129
dtype: float64

In [26]:
# # We create the train and test sets using train_test_split
# train_x, test_x, train_y, test_y = train_test_split(bow,target, test_size=0.20 ,random_state=0)

## Naïve Bayes & Logistic regression

We define NaÏve Bayes relation

In [27]:
def probNB(bow,target,cat):

    '''
    Naive Bayes probability for each word
    Inputs :
    bow : bag of words (with doc in rows and words in columns)
    target : classification vector (filled with 1 and 0)
    cat : 1 or 0, in target
    Output : 
    Vector of Naive Bayes probabilities with smoothing (n_words,1)
    '''

    p = np.array(bow[target==cat].sum(axis=0))

    return np.transpose((p+1) / (p.sum() + bow.shape[1]))
    


In [28]:
def get_model(bow,target):

    '''
    Function that return the log likelihood of a document
    Inputs :
    bow : bag of words (n_doc,n_words)
    target : classification of comments (n_doc,1)
    Output : 
    Return a vector of Log Likelihood for each comment (Naïve Bayes) (n_doc,1)
    '''

    log = np.log(probNB(bow,target,1)/probNB(bow,target,0))
    m = bow.dot(log)
    model = LogisticRegression().fit(m,target)
    return model , log

In [29]:
df_classification = pd.DataFrame() #We store probabilities into a Dataframe
df_classification['Comments'] = test['comment_text']

for i,j in enumerate(target.columns):
    print('fit', j)
    model,log = get_model(bow,target[j])
    df_classification[j] = model.predict(bow_test.dot(log))

    #Acurracy
    score = model.score(bow_test.dot(log) , test[j])
    print(f"Accuracy : {score:.4}")
    print('----')   

fit toxic
Accuracy : 0.9279
----
fit severe_toxic
Accuracy : 0.9938
----
fit obscene
Accuracy : 0.9507
----
fit threat
Accuracy : 0.9961
----
fit insult
Accuracy : 0.9503
----
fit identity_hate
Accuracy : 0.9885
----


In [30]:
target.columns

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

### Let's see where our model was wrong

Apply the algorithm to all comments


Let's look at the differences between our model results and the target

In [387]:
keys = target.columns

# List of all comments where our model was wrong 
# We filter any row that have a difference with the target, located in test

# df of True or False for each comment (rows) and class (columns), True means there is a difference
df_comparison = df_classification[keys] != test[keys] 

# We use pd.dataframe.any(axis='columns') that returns True if all True, or False otherwise
# In other word, if there is a difference, it returns True for the entire row
error_insult_comments = df_classification.loc[df_comparison.any(axis=1)]['Comments']

# Get the index list of mismatches
index_list = error_insult_comments.index

for i in np.random.choice(index_list,10):
    print(error_insult_comments[i])
    print('---------')
    print(f'Classes {list(keys)}')
    print(f"Actual {[test[key][i] for key in keys]}")
    print(f'Model {[df_classification[key][i] for key in keys]}')
    print()


lesbian bizzle who knows no shizzle
---------
Classes ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
Actual [1, 0, 0, 0, 1, 1]
Model [0, 0, 0, 0, 0, 0]

::::Fuck. I misread the dates. Umm. Sorry.
---------
Classes ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
Actual [1, 0, 1, 0, 0, 0]
Model [0, 0, 0, 0, 0, 0]

How dare you say I vandalize! >=( you will PAY! $W%$##! #$%^&!! #^&*(!! $%!@@#@@#! You !@##@@ homosexual gay fag homo gay homo
---------
Classes ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
Actual [1, 0, 1, 0, 1, 1]
Model [1, 0, 0, 0, 0, 0]

i fucked your mum
---------
Classes ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
Actual [1, 0, 1, 0, 0, 0]
Model [1, 0, 0, 0, 0, 0]

== THIS IS CHARLIE 12 27 == 

 STOP BANNING ME FAGGS!!!!!!!!!!!!! IM THE COOOLESTE AND YORE ALLGAY FUCKES!!!!! SO GO TE HELL!!!!!!!!!!!!!!11111LOOSER S !!!!!!! 111
---------
Classes ['toxic', 'severe_

**Notes**  

As we can, our model misses a lot of classification.  
Let's calculate the overall accuracy, which means that any missmatch for a comment is counted as wrong.

In [388]:
overall_accuracy = df_comparison.any(axis=1).sum(axis=0) / df_comparison.shape[0]
print(f'Overall accuracy is : {overall_accuracy:.3}')

Overall accuracy is : 0.107


## TF-IDF NB-Logistic regression

In [389]:
# Word embeddings
tfidf_vec = TfidfVectorizer(min_df=1,max_df=0.9)
tfidf = tfidf_vec.fit_transform(clean_comments)
tfidf_test = tfidf_vec.transform(test_clean_comments)

# # Split data
# # We create the train and test sets using train_test_split
# train_x, test_x, train_y, test_y = train_test_split(tfidf,target, test_size=0.20 ,random_state=0)

# Let's create our model and calculate accuracy
df_classification = pd.DataFrame() #We store probabilities into a Dataframe
df_classification['Comments'] = test['comment_text']

for i,j in enumerate(target.columns):
    print('fit', j)
    model,log = get_model(tfidf,target[j])
    df_classification[j] = model.predict(tfidf_test.dot(log))

    #Acurracy
    score = model.score(tfidf_test.dot(log) , test[j])
    print(f"Accuracy : {score:.4}")
    print('----')   


fit toxic
Accuracy : 0.9162
----
fit severe_toxic
Accuracy : 0.9942
----
fit obscene
Accuracy : 0.9482
----
fit threat
Accuracy : 0.9967
----
fit insult
Accuracy : 0.9471
----
fit identity_hate
Accuracy : 0.9889
----


### With Bag Of Words embedding :

fit toxic
Accuracy : 0.9269

fit severe_toxic
Accuracy : 0.9931

fit obscene
Accuracy : 0.9472

fit threat
Accuracy : 0.9962

fit insult
Accuracy : 0.9476

fit identity_hate
Accuracy : 0.9883

**Note**  
We slightly increase the performance overall using TfIdf embedding, except for toxic class

In [390]:
keys = target.columns

# List of all comments where our model was wrong 
# We filter any row that have a difference with the target, located in test

# df of True or False for each comment (rows) and class (columns), True means there is a difference
df_comparison = df_classification[keys] != test[keys] 

# We use pd.dataframe.any(axis='columns') that returns True if all True, or False otherwise
# In other word, if there is a difference, it returns True for the entire row
error_insult_comments = df_classification.loc[df_comparison.any(axis=1)]['Comments']

# Get the index list of mismatches
index_list = error_insult_comments.index

for i in np.random.choice(index_list,5):
    print(error_insult_comments[i])
    print('---------')
    print(f'Classes {list(keys)}')
    print(f"Actual {[test[key][i] for key in keys]}")
    print(f'Model {[df_classification[key][i] for key in keys]}')
    print()

"""Fuck Dre, tell that bitch he can kiss my ass"" 

 http://www.pacandbig.com/2pac/lyrics/Until%20The%20End%20Of%20Time/CD1/02%20-%20Fuck%20Friendz.txt 

 ""No longer Dre Day, arrivederci 
 Blown and forgotten, rotten for plottin Child's Play 
 Check your sexuality, as fruity as this Alize 
 Quick to jump ship, punk trick, what a dumb move 
 Cross Death Row, now who you gon' run to? 
 Lookin for suckers cause you similar 
 Pretendin to be hard, oh my God, check your temperature 
 Screamin Compton, but you can't return, you ain't heard 
 Brothers pissed cause you switched and escaped to the burbs 
 Mob on to this new era, cause we Untouchable 
 Still can't believe that you got 'Pac rushin you 
 Up in you, bless the real, all the rest get killed 
 Who can you trust, only time reveals  toss it up!"" 

 http://www.pacandbig.com/2pac/lyrics/The%20Don%20Killuminati%20The%207%20Day%20Theory/03%20-%20Toss%20It%20Up.txt 

 ""LA, California Love part motherfuckin Two 
 Without gay ass Dre"" 

 h

In [391]:
overall_accuracy = df_comparison.any(axis=1).sum(axis=0) / df_comparison.shape[0]
print(f'Overall accuracy is : {overall_accuracy:.3}')

Overall accuracy is : 0.118


## Conclusions on models

As we can see, if we check the overall accuracy, by considering all classification at the same time, it appears that our model perfoms badly.

* 10,7% correct accuracy for BOW + NB-Logistic regression
* 11,8% for TF-IDF + Logistic Regression

If we look at each classification individually, we note :

* 96.1% accuracy in average for BOW
* 96.7% accuracy in average for TF-IDF

*UPDATE*  
After submission on Kaggle the score of TFIDF NB Logistic Regression model is 0.84586!

In [396]:
# Calculate classes accuracy mean for bow and TF-IDF
np.mean([0.9269,0.9931,0.9472,0.9476,0.9883]) , np.mean([0.9269,0.9931,0.9472,0.9962,0.9476,0.9883])

(0.9606199999999999, 0.9665499999999999)

Let's save models for later use 

In [ ]:
# If you need to run this code, make the statment true
if 0==1:

    # TFIDF vectorizer
    pickle.dump(tfidf_vec,open('tfidf_vectorizer.pkl','wb'))

    # Logistic regression models + Log Likelihood based on Naïve Bayes for each toxicity classification
    for i,j in enumerate(target.columns):
        print('fit', j)
        model,log = get_model(tfidf,target[j])
        pickle.dump((model,log),open('models_'+f'{target.columns[i]}.pkl','wb')) #Save models and logs as tuples

# Deployment

Let's define an algorithm that proceeds a comment and return classification of it

In [ ]:
# From the folder Models, we import the Logistic model and the Log Likelihood
for dirname, _, filenames in os.walk('Models'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def get_toxicity(comment):

    '''
    A function that takes a raw comment as input and returns the a vector of categorical 
    values as ['identity_hate','insult','obscene','severe_toxic','threat','toxic'].
    '''

    # Get everything we need for this function
    from utils import preprocess
    tfidf_vec = pickle.load(open("tfidf_vectorizer.pkl",'rb'))


    # Declaration of comment classification as a pd.Series
    classification = pd.Series(index=['identity_hate','insult','obscene','severe_toxic','threat','toxic'],dtype=int)
    i = 0 # We be used as increment to fill the vector

    # Clean comment
    clean_comments = preprocess(comment)

    # Get word embeddings
    word_emb = tfidf_vec.transform(clean_comments)

    # From the folder Models, we import the Logistic model and the Log Likelihood, one by one
    for dirname, _, filenames in os.walk('Models'):
        for filename in filenames:
            path = os.path.join(dirname, filename)
            model,log = pickle.load(open(path,'rb'))

            classification[i] = model.predict(word_emb.dot(log))
            i += 1

    return classification

In [ ]:
comment = ["Stupid chinese people!"]
get_toxicity(comment)

In [ ]:
from utils import preprocess

preprocess(comment)